This file judges if a galaxy has its mms galaxy on the first approach and plots the figures related to the judgement.

In [ ]:
import os
import gc
from tqdm import tqdm
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits import mplot3d
import math

# %matplotlib widget
basePath = "./sims.TNG/TNG100-1/output"


In [ ]:
initial_snapshot = 69
end_snapshot = 99

halos_fields = ["GroupFirstSub", "GroupNsubs", "GroupMass", "GroupPos"]
subhalos_fields = ["SubhaloGrNr", "SubhaloMass", "SubhaloPos"]
tree_fields = ["SubfindID", "SnapNum"]

h = 0.6774  # km/Mpc

m_dm = 4.5e-5  # E10M_Sum
m_dm_h = m_dm * h

Critial_Density = 1.27e-8  # E10M_Sum/kpc^3
Critial_Density_h = Critial_Density / h**2  # E10M_Sum/(kpc/h)^3


In [ ]:
halos_data = il.groupcat.loadHalos(basePath, end_snapshot, fields=halos_fields)
subhalos_data = il.groupcat.loadSubhalos(basePath, end_snapshot, fields=subhalos_fields)


In [ ]:
first_subhalo_indices = halos_data["GroupFirstSub"]
n_subhalos = halos_data["GroupNsubs"]
group_mass = halos_data["GroupMass"]
subhalos_grnr = subhalos_data["SubhaloGrNr"]
subhalos_mass = subhalos_data["SubhaloMass"]


In [ ]:
galaxy_mass_inf = 30
galaxy_mass_sup = 500
condition = (group_mass >= galaxy_mass_inf) & (group_mass <= galaxy_mass_sup)
mw_like_group_index = np.where(condition)[0]


In [ ]:
mw_like_group_index = np.load("./MW_Like_Indices.npy")


In [ ]:
def Calculate_Radial_Velocity(halo_index):
    halo_dict = il.groupcat.loadSingle(basePath, end_snapshot, halo_index, -1)
    if halo_dict['GroupFirstSub'] == -1:
        return -1
    if halo_dict['GroupNsubs'] == -1:
        return -1

    subhalos_masses_list = list()

    for subhalo_index in range(halo_dict['GroupFirstSub'], halo_dict['GroupFirstSub']+halo_dict['GroupNsubs']):
        subhalos_masses_list.append(subhalos_mass[subhalo_index])

    subhalos_masses_array = np.array(subhalos_masses_list)

    del subhalos_masses_list
    gc.collect()

    host_index = halo_dict['GroupFirstSub'] + \
        np.argsort(subhalos_masses_array)[-1]
    mms_index = halo_dict['GroupFirstSub'] + \
        np.argsort(subhalos_masses_array)[-2]

    host_tree = il.sublink.loadTree(
        basePath, end_snapshot, host_index, fields=tree_fields, onlyMPB=1)
    mms_tree = il.sublink.loadTree(
        basePath, end_snapshot, mms_index, fields=tree_fields, onlyMPB=1)

    satellite_velocity_list = list()
    host_mms_distance = list()
    host_position_list = list()
    mms_position_list = list()

    for current_snapshot in range(initial_snapshot, end_snapshot+1):

        header = il.groupcat.loadHeader(basePath, current_snapshot)
        scale_factor = header['Time']
        host_subhalo_index = host_tree['SubfindID'][np.where(
            host_tree['SnapNum'] == current_snapshot)[0]]
        mms_subhalo_index = mms_tree['SubfindID'][np.where(
            mms_tree['SnapNum'] == current_snapshot)[0]]

        if len(mms_subhalo_index) == 0 or len(host_subhalo_index) == 0:
            satellite_velocity_list.append(0)
            host_mms_distance.append(0)
            host_position_list.append(np.array([0, 0, 0]))
            mms_position_list.append(np.array([0, 0, 0]))
            continue

        host_subhalo_dict = il.groupcat.loadSingle(
            basePath, current_snapshot, -1, host_subhalo_index)
        mms_subhalo_dict = il.groupcat.loadSingle(
            basePath, current_snapshot, -1, mms_subhalo_index)

        current_host_comoving_position = host_subhalo_dict['SubhaloPos']
        current_mms_comoving_position = mms_subhalo_dict['SubhaloPos']

        current_host_physical_position = current_host_comoving_position*scale_factor
        current_mms_physical_position = current_mms_comoving_position*scale_factor

        current_host_physical_position = current_host_physical_position % 35000
        current_mms_physical_position = current_mms_physical_position % 35000

        current_difference_physical_coordinates = current_mms_physical_position - \
            current_host_physical_position
        distance = np.linalg.norm(current_difference_physical_coordinates)

        host_velocity = host_subhalo_dict['SubhaloVel']
        mms_velocity = mms_subhalo_dict['SubhaloVel']

        Vel_diff = mms_velocity-host_velocity
        Vel_Radius = np.dot(
            Vel_diff, current_difference_physical_coordinates)/distance

        satellite_velocity_list.append(Vel_Radius)
        host_mms_distance.append(distance)

        host_position_list.append(current_host_physical_position)
        mms_position_list.append(current_mms_physical_position)

        gc.collect()
    satellite_radius_velocity = np.array(satellite_velocity_list)
    host_mms_distance = np.array(host_mms_distance)
    host_position_array = np.array(host_position_list)
    mms_position_array = np.array(mms_position_list)

    return_data = dict()
    return_data['Host_Index'] = host_index
    return_data['MMS_Index'] = mms_index
    return_data['Halo_Index'] = halo_index
    return_data['RadialVelocity'] = satellite_radius_velocity
    return_data['Distance'] = host_mms_distance
    return_data['HostPosition'] = host_position_array
    return_data['MMSPosition'] = mms_position_array
    return_data['Snapshots'] = np.arange(initial_snapshot, end_snapshot+1)
    return return_data


def Distance_Plot(Halo_Dict, SavePath, Virial_Radius=None):
    halo_index = Halo_Dict['Halo_Index']
    plt.figure(figsize=(8, 6))
    plt.plot(Halo_Dict['Snapshots'], Halo_Dict['Distance'])
    plt.grid()
    plt.xlabel('Snapshot')
    plt.ylabel('Satellite-Host Distance (kpc/h)')
    plt.title(f'Distance Evolution of Halo-{halo_index}')
    if Virial_Radius is not None:
        # Add a horizontal dashed line to represent the Virial Radius
        plt.axhline(y=Virial_Radius, color='red',
                    linestyle='--', label='Virial Radius')
    plt.savefig(os.path.join(
        SavePath, f'Halo-{halo_index} Distance Evolution'))
    plt.close('all')
    gc.collect()


def OrbitTrajectory3D_Plot(Halo_Dict, SavePath):
    halo_index = Halo_Dict['Halo_Index']
    fig = plt.figure(figsize=(10, 6))
    ax = plt.axes(projection='3d')
    ax.set_box_aspect([1, 1, 1])
    ax.grid(True, linestyle='--', alpha=0.5)

    x = Halo_Dict['MMSPosition'][:, 0]-Halo_Dict['HostPosition'][:, 0]
    y = Halo_Dict['MMSPosition'][:, 1]-Halo_Dict['HostPosition'][:, 1]
    z = Halo_Dict['MMSPosition'][:, 2]-Halo_Dict['HostPosition'][:, 2]
    ax.plot3D(x, y, z)
    ax.set_title(f'Halo-{halo_index} Orbit Trajectory')
    ax.scatter(0, 0, 0, color='black', s=20)
    max_range = max(max(x), max(y), max(z))
    min_range = min(min(x), min(y), min(z))
    ax.set_xlim(min_range, max_range)
    ax.set_ylim(min_range, max_range)
    ax.set_zlim(min_range, max_range)
    ax.set_xlabel('X (kpc/h)')
    ax.set_ylabel('Y (kpc/h)')
    ax.set_zlabel('Z (kpc/h)')
    for i in range(initial_snapshot, initial_snapshot+len(x), 5):
        ax.text(x[i-initial_snapshot], y[i-initial_snapshot],
                z[i-initial_snapshot], f'Snap-{i}', color='black', fontsize=10)
    plt.savefig(os.path.join(SavePath, f'Halo-{halo_index} Orbit Trajectory'))
    plt.close('all')
    gc.collect()


def Orbit_Trajectory_Plot(Array1, Array2, Label1, Label2, SavePath, Halo_Index, Virial_Radius):
    plt.figure(figsize=(8, 6))
    plt.plot(Array1, Array2)
    plt.scatter(0, 0, c='black', s=20)

    # Add a red dashed circle with Virial Radius
    circle = plt.Circle((0, 0), Virial_Radius, color='red',
                        fill=False, linestyle='--')
    plt.gca().add_patch(circle)

    plt.gca().set_aspect('equal', adjustable='box')
    plt.grid()
    plt.xlabel(Label1 + ' (kpc/h)')
    plt.ylabel(Label2 + ' (kpc/h)')
    plt.title(f'Halo-{Halo_Index} ' + Label1 + '-' +
              Label2 + ' Plane Orbit Trajectory')

    for i in range(initial_snapshot, initial_snapshot + len(Array1), 5):
        plt.text(Array1[i - initial_snapshot], Array2[i - initial_snapshot],
                 f'Snap-{i}', color='black', fontsize=9)

    # Annotate Virial Radius
    plt.text(Virial_Radius, 0, f'Virial Radius', color='red',
             fontsize=9, verticalalignment='bottom')

    plt.savefig(os.path.join(
        SavePath, f'Halo-{Halo_Index} ' + Label1 + '-' + Label2 + ' Plane Orbit Trajectory'))
    plt.close('all')
    gc.collect()


def RadialVelocity_Plot(Halo_Dict, SavePath):
    halo_index = Halo_Dict['Halo_Index']
    plt.figure(figsize=(8, 6))
    plt.plot(Halo_Dict['Snapshots'], Halo_Dict['RadialVelocity'])
    plt.grid()
    plt.xlabel('Snapshot')
    plt.ylabel('MMS Radial Velocity (km/s)')
    plt.title(f'Radial Velocity Evolution of Halo-{halo_index}')
    plt.axhline(0, color='red', linestyle='--')  # 添加y=0的水平线
    plt.savefig(os.path.join(
        SavePath, f'Halo-{halo_index} Radial Velocity Evolution'))
    plt.close('all')
    gc.collect()


def OrbitTrajectory2D_Plot(Halo_Dict, SavePath, Virial_Radius):
    halo_index = Halo_Dict['Halo_Index']
    x = Halo_Dict['MMSPosition'][:, 0]-Halo_Dict['HostPosition'][:, 0]
    y = Halo_Dict['MMSPosition'][:, 1]-Halo_Dict['HostPosition'][:, 1]
    z = Halo_Dict['MMSPosition'][:, 2]-Halo_Dict['HostPosition'][:, 2]

    Orbit_Trajectory_Plot(x, y, 'X', 'Y', Detect_Mkdir(
        os.path.join(SavePath, 'XY-Trajectory')), halo_index, Virial_Radius)
    Orbit_Trajectory_Plot(y, z, 'Y', 'Z', Detect_Mkdir(
        os.path.join(SavePath, 'YZ-Trajectory')), halo_index, Virial_Radius)
    Orbit_Trajectory_Plot(x, z, 'X', 'Z', Detect_Mkdir(
        os.path.join(SavePath, 'XZ-Trajectory')), halo_index, Virial_Radius)

    gc.collect()


def Detect_Mkdir(Path):
    if not (os.path.exists(Path)):
        os.makedirs(Path)
    return Path


def R_200_Calculation(Snapshot_Num, Subhalo_Id):
    subhalo_data_dict = il.groupcat.loadSingle(
        basePath, Snapshot_Num, -1, Subhalo_Id)

    subhalo_comoving_position_floatarray = subhalo_data_dict["SubhaloPos"]

    gas_data_dict = il.snapshot.loadSubhalo(
        basePath, Snapshot_Num, Subhalo_Id, "gas", fields=["Coordinates", "Masses"]
    )
    darkmatter_data_dict = il.snapshot.loadSubhalo(
        basePath, Snapshot_Num, Subhalo_Id, "darkmatter", fields=["Coordinates"]
    )
    stellar_data_dict = il.snapshot.loadSubhalo(
        basePath, Snapshot_Num, Subhalo_Id, "stellar", fields=["Coordinates", "Masses"]
    )
    blackholes_data_dict = il.snapshot.loadSubhalo(
        basePath,
        Snapshot_Num,
        Subhalo_Id,
        "blackholes",
        fields=["Coordinates", "Masses"],
    )

    precision = 1
    max_iter = 100
    left, right = 0, 500

    header_dict = il.groupcat.loadHeader(basePath, Snapshot_Num)

    scale_factor = header_dict["Time"]

    for i in range(max_iter):
        mid = (left + right) / 2

        mass_in_radius = Total_Mass_Within_Given_Radius(
            subhalo_comoving_position_floatarray,
            mid,
            gas_data_dict,
            darkmatter_data_dict,
            stellar_data_dict,
            blackholes_data_dict,
        )

        mass_200 = Cal_M_From_R_Phy(mid * scale_factor, header_dict)

        if mass_in_radius > mass_200:
            left = mid

        else:
            right = mid

        if abs(right - left) < precision or i == max_iter - 1:
            R_200_Com_Float = (left + right) / 2
            break

    R_200_Phy_Float = R_200_Com_Float * scale_factor

    M_200_Float = Cal_M_From_R_Phy(R_200_Phy_Float, header_dict)

    del (
        subhalo_data_dict,
        subhalo_comoving_position_floatarray,
        gas_data_dict,
        darkmatter_data_dict,
        stellar_data_dict,
        blackholes_data_dict,
        precision,
        max_iter,
        left,
        right,
        header_dict,
        scale_factor,
        mid,
        mass_in_radius,
        mass_200,
    )

    gc.collect()
    return R_200_Phy_Float, M_200_Float


def Cal_M_From_R_Phy(R_200_Float, Header_Dict):
    Omega_m = Header_Dict["Omega0"]
    Omega_Lambda = Header_Dict["OmegaLambda"]
    z = Header_Dict["Redshift"]
    scale_factor = Header_Dict["Time"]

    E_z = Omega_m * (1 + z) ** 3 + Omega_Lambda
    M_200_Float = 200 * Critial_Density_h * \
        E_z * (R_200_Float**3) * 4 / 3 * 3.14

    del Omega_m, Omega_Lambda, z, scale_factor, E_z

    return M_200_Float


def Baryon_Mass_Within_Given_Radius(
    Galaxy_Center_FloatArray,
    Mass_Radius_Float,
    Particle_Coordinates_FloatArray,
    Particle_Masses_FloatArray,
):
    dist_to_com_floatarray = np.sqrt(
        np.sum(
            (Particle_Coordinates_FloatArray - Galaxy_Center_FloatArray) ** 2, axis=1
        )
    )
    within_radius_indices_intarray = np.where(
        dist_to_com_floatarray <= Mass_Radius_Float
    )[0]
    Total_Mass_Within_Radius_Float = np.sum(
        Particle_Masses_FloatArray[within_radius_indices_intarray]
    )

    del dist_to_com_floatarray, within_radius_indices_intarray

    return Total_Mass_Within_Radius_Float


def DarkMatter_Mass_Within_Given_Radius(
    Galaxy_Center_FloatArray, Mass_Radius_Float, Particle_Coordinates_FloatArray
):
    dist_to_com_floatarray = np.sqrt(
        np.sum(
            (Particle_Coordinates_FloatArray - Galaxy_Center_FloatArray) ** 2, axis=1
        )
    )
    within_radius_indices_intarray = np.where(
        dist_to_com_floatarray <= Mass_Radius_Float
    )[0]
    Total_Mass_Within_Radius_Float = m_dm_h * \
        within_radius_indices_intarray.shape[0]

    del dist_to_com_floatarray, within_radius_indices_intarray

    return Total_Mass_Within_Radius_Float


def Total_Mass_Within_Given_Radius(
    Galaxy_Center_FloatArray,
    Mass_Radius_Float,
    Gas_Data_Dict,
    DarkMatter_Data_Dict,
    Stellar_Data_Dict,
    BlackHole_Data_Dict,
):
    darkmatter_mass_within_radius = DarkMatter_Mass_Within_Given_Radius(
        Galaxy_Center_FloatArray, Mass_Radius_Float, DarkMatter_Data_Dict
    )

    if len(Gas_Data_Dict) == 1:
        gas_mass_within_radius = 0
    else:
        gas_mass_within_radius = Baryon_Mass_Within_Given_Radius(
            Galaxy_Center_FloatArray,
            Mass_Radius_Float,
            Gas_Data_Dict["Coordinates"],
            Gas_Data_Dict["Masses"],
        )

    if len(Stellar_Data_Dict) == 1:
        stellar_mass_within_radius = 0
    else:
        stellar_mass_within_radius = Baryon_Mass_Within_Given_Radius(
            Galaxy_Center_FloatArray,
            Mass_Radius_Float,
            Stellar_Data_Dict["Coordinates"],
            Stellar_Data_Dict["Masses"],
        )

    if len(BlackHole_Data_Dict) == 1:
        blackholes_mass_within_radius = 0
    else:
        blackholes_mass_within_radius = Baryon_Mass_Within_Given_Radius(
            Galaxy_Center_FloatArray,
            Mass_Radius_Float,
            BlackHole_Data_Dict["Coordinates"],
            BlackHole_Data_Dict["Masses"],
        )

    Total_Mass_Within = (
        gas_mass_within_radius
        + darkmatter_mass_within_radius
        + stellar_mass_within_radius
        + blackholes_mass_within_radius
    )

    del (
        gas_mass_within_radius,
        darkmatter_mass_within_radius,
        stellar_mass_within_radius,
        blackholes_mass_within_radius,
    )
    gc.collect()
    return Total_Mass_Within


def Cal_Subhalo_Id_at_Z_from_99(Subhalo_Id_0, Z):  # Z is the snapshot
    subfindid_snapnum_tree = il.sublink.loadTree(
        basePath,
        99,
        Subhalo_Id_0,
        fields=["SubfindID", "SnapNum", "Mass"],
        onlyMPB=True,
    )

    if subfindid_snapnum_tree == None:
        return -1

    subfind_id = subfindid_snapnum_tree["SubfindID"]
    snapshot_num = subfindid_snapnum_tree["SnapNum"]
    index = np.where(snapshot_num == Z)[0]
    if index.shape[0] == 0:
        return -1
    return subfind_id[index][0]


In [ ]:
SavePath = "./MMS_TNG100/"
Detect_Mkdir(SavePath)
distance_path = os.path.join(SavePath, "Distance_Evolution")
orbit3D_path = os.path.join(SavePath, "Orbit_Trajectory3D")
orbitXY_path = os.path.join(SavePath, "XY-Trajectory")
orbitXZ_path = os.path.join(SavePath, "XZ-Trajectory")
orbitYZ_path = os.path.join(SavePath, "YZ-Trajectory")
velocity_path = os.path.join(SavePath, "Radial_Velocity")


In [ ]:
##别运行

FilePath = Detect_Mkdir(os.path.join('./TNG100_Final_File'))
##if os.path.exists(FilePath):
##    for file in os.listdir(FilePath):
##        file = os.path.join(FilePath, file)
##        if os.path.isfile(file):
##            os.remove(file)
##    os.rmdir(FilePath)


In [ ]:
FilePath = Detect_Mkdir(os.path.join('./TNG100_Final_File'))
Final_File_List = ['Halo_Array', 'Host_Array', 'MMS_Array', 'First_App_Snap_Array', 'First_App_Distance_Array',
                   'Snap99_Distance_Array', 'Snap99_Radial_Velocity_Array', 'Snap99_Virial_Radius_Array', 'Is_First_Approach']
for filename in Final_File_List:
    if os.path.exists(os.path.join(FilePath, filename)+'.npy'):
        globals()[filename] = np.load(
            (os.path.join(FilePath, filename)+'.npy'))

start_sub_index = Halo_Array.shape[0]
step_length = 5
end = start_sub_index+step_length
if end > mw_like_group_index.shape[0]:end =mw_like_group_index.shape[0]

Final_File_Halo_Index = list()
Final_File_Host_Index = list()
Final_File_MMS_Index = list()
Final_File_First_App_Snap = list()
Final_File_First_App_Distance = list()
Final_File_Snap99_Distance = list()
Final_File_Snap99_Radial_Velocity = list()
Final_File_Snap99_Virial_Radius = list()
Final_File_Is_First_Approach = list()

for Halo_Index in tqdm(mw_like_group_index[start_sub_index:end]):
    halo_dict = Calculate_Radial_Velocity(Halo_Index)
    VirialRadius, VirialMass = R_200_Calculation(
        end_snapshot, halo_dict['Host_Index'])

    Distance_Plot(halo_dict, Detect_Mkdir(distance_path), VirialRadius)
    OrbitTrajectory3D_Plot(halo_dict, Detect_Mkdir(orbit3D_path))
    OrbitTrajectory2D_Plot(halo_dict, SavePath, VirialRadius)
    RadialVelocity_Plot(halo_dict, Detect_Mkdir(velocity_path))

    If_First_App_At_99 = (np.argsort(halo_dict['Distance'])[
        0] == (end_snapshot-initial_snapshot))
    If_Radial_Velocity_Negative = (
        halo_dict['RadialVelocity'][end_snapshot-initial_snapshot] < 0)
    If_In_Virial_Radius = (
        halo_dict['Distance'][end_snapshot-initial_snapshot] < VirialRadius)
    If_First_Approach = If_First_App_At_99 and If_Radial_Velocity_Negative and If_In_Virial_Radius

    Final_File_Halo_Index.append(Halo_Index)
    Final_File_Host_Index.append(halo_dict['Host_Index'])
    Final_File_MMS_Index.append(halo_dict['MMS_Index'])
    Final_File_First_App_Snap.append(np.argsort(
        halo_dict['Distance'])[0]+initial_snapshot)
    Final_File_First_App_Distance.append(
        halo_dict['Distance'][np.argsort(halo_dict['Distance'])[0]])
    Final_File_Snap99_Distance.append(
        halo_dict['Distance'][end_snapshot-initial_snapshot])
    Final_File_Snap99_Radial_Velocity.append(
        halo_dict['RadialVelocity'][end_snapshot-initial_snapshot])
    Final_File_Snap99_Virial_Radius.append(VirialRadius)
    Final_File_Is_First_Approach.append(int(If_First_Approach))

    gc.collect()

Final_File_Halo_Array = np.array(Final_File_Halo_Index)
Final_File_Host_Array = np.array(Final_File_Host_Index)
Final_File_MMS_Array = np.array(Final_File_MMS_Index)
Final_File_First_App_Snap_Array = np.array(Final_File_First_App_Snap)
Final_File_First_App_Distance_Array = np.array(
    Final_File_First_App_Distance)
Final_File_Snap99_Distance_Array = np.array(Final_File_Snap99_Distance)
Final_File_Snap99_Radial_Velocity_Array = np.array(
    Final_File_Snap99_Radial_Velocity)
Final_File_Snap99_Virial_Radius_Array = np.array(
    Final_File_Snap99_Virial_Radius)
Final_File_Is_First_Approach_Array = np.array(Final_File_Is_First_Approach)


Final_File_List = ['Halo_Array', 'Host_Array', 'MMS_Array', 'First_App_Snap_Array', 'First_App_Distance_Array',
                   'Snap99_Distance_Array', 'Snap99_Radial_Velocity_Array', 'Snap99_Virial_Radius_Array', 'Is_First_Approach']
for filename in Final_File_List:
    if os.path.exists(os.path.join(FilePath, filename)+'.npy'):
        Previous_File = np.load((os.path.join(FilePath, filename)+'.npy'))
        Next_File_Name = 'Final_File_'+filename
        Next_File_Array = globals()[Next_File_Name]
        ConCated_File = np.concatenate(
            (Previous_File, Next_File_Array), axis=0)
        np.save((os.path.join(FilePath, filename)+'.npy'), ConCated_File)
    else:
        File_Name = 'Final_File_'+filename
        File_Array = globals()[File_Name]
        np.save((os.path.join(FilePath, filename)+'.npy'), File_Array)


In [ ]:
FilePath = Detect_Mkdir(os.path.join('./TNG100_Final_File'))
Final_File_List = ['Halo_Array', 'Host_Array', 'MMS_Array', 'First_App_Snap_Array', 'First_App_Distance_Array',
                   'Snap99_Distance_Array', 'Snap99_Radial_Velocity_Array', 'Snap99_Virial_Radius_Array', 'Is_First_Approach']
for filename in Final_File_List:
    if os.path.exists(os.path.join(FilePath, filename)+'.npy'):
        globals()[filename] = np.load(
            (os.path.join(FilePath, filename)+'.npy'))
        print(globals()[filename].shape[0])


In [ ]:
np.savez(os.path.join(FilePath, 'AllData.npz'), Halo_Index=Final_File_Halo_Array, Host_Subhalo_Index=Final_File_Host_Array, MMS_Subhalo_Index=Final_File_MMS_Array, First_App_Snapshot=Final_File_First_App_Snap_Array, First_App_Distance=Final_File_First_App_Distance_Array,
         Distance_At_Snap99=Final_File_Snap99_Distance_Array, Radial_Velocity_At_Snap99=Final_File_Snap99_Radial_Velocity_Array, Virial_Radius_At_Snap99=Final_File_Snap99_Virial_Radius_Array, Is_On_First_Approach=Final_File_Is_First_Approach_Array)
